# Generate CONUS LFMC Image
#### Description
Creates a GeoTiff image of LFMC predictions that can be used to produce LFMC maps.

#### Input Images
1. An image of auxiliary data - latitude, longitude, elevation, slope, aspect and climate zone
2. Images of MODIS data for at least 1 year prior to the mapping date
3. Images of PRISM data for at least 1 year prior to the mapping date  
Note: Band names for PRISM data are confusing. When GEE converts an image collection to an image, it includes the date in the band name. Timestamps on PRISM data are midday, so when they are converted (rounded) to a date, this becomes the following day. So in the extracted GeoTiffs, bands for 2016-10-01 data will named for 2016-10-02!

#### Other Inputs
1. Model directory - this should contain "run" directories - one for each model in the ensemble.
2. Data used to train the model - The notebook extracts normalisation bounds and one-hot encodings needed to prepare the input data. The normalisation bounds are saved to csv files, so if these files already exist, bounds can be loaded from these instead.
3. Legend file for Koppen climate zones. This should be a CSV as created by the "Extract Auxiliary Data.ipynb" notebook. Used to convert the climate zone numbers in the auxiliary input into climate zone codes


In [1]:
import glob
import numpy as np
import os
import pandas as pd
import time

from osgeo import gdal

In [2]:
import initialise
import common
from display_utils import display_frames

In [3]:
KOPPEN_LEGEND = os.path.join(common.SOURCE_DIR, 'Climate_zones.csv')
czones = pd.read_csv(KOPPEN_LEGEND, index_col=0)

In [4]:
april_file = os.path.join(common.MAPS_DIR, 'LFMC_maps', 'LFMC_difference_2018-04-01.tif')
october_file = os.path.join(common.MAPS_DIR, 'LFMC_maps', 'LFMC_difference_2018-10-01.tif')

In [5]:
april_image = gdal.Open(april_file, gdal.GA_ReadOnly)
april_data = april_image.ReadAsArray()
april_data = april_data.flatten()[(april_data.flatten() > -10000)]

In [6]:
april_diffs = pd.Series(april_data).abs()

In [7]:
april_diffs.quantile([.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

0.1     1.100246
0.2     2.231766
0.3     3.431123
0.4     4.744949
0.5     6.222733
0.6     7.868958
0.7     9.725439
0.8    11.871115
0.9    14.875572
1.0    71.509903
dtype: float64

In [8]:
(april_diffs.le(10).sum()/april_diffs.count()).round(2)

0.71

In [9]:
october_image = gdal.Open(october_file, gdal.GA_ReadOnly)
october_data = october_image.ReadAsArray()
october_data = october_data.flatten()[(october_data.flatten() > -10000)]
october_diffs = pd.Series(october_data).abs()

In [10]:
october_diffs.quantile([.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

0.1     0.842575
0.2     1.708359
0.3     2.618912
0.4     3.610596
0.5     4.724976
0.6     6.033585
0.7     7.639532
0.8     9.761243
0.9    13.123604
1.0    62.081757
dtype: float64

In [11]:
(october_diffs.le(10).sum()/october_diffs.count()).round(2)

0.81

In [12]:
(april_diffs.le(5).sum()/april_diffs.count()).round(2)

0.42

In [13]:
(october_diffs.le(5).sum()/october_diffs.count()).round(2)

0.52

In [14]:
april_proj_file = os.path.join(common.MAPS_DIR, 'LFMC_maps', 'Projection_base_2018-04-01.tif')
october_proj_file = os.path.join(common.MAPS_DIR, 'LFMC_maps', 'Projection_base_2018-10-01.tif')

In [15]:
april_image = gdal.Open(april_proj_file, gdal.GA_ReadOnly)
april_data = april_image.ReadAsArray()[0]
april_projections = pd.Series(april_data.flatten()[(april_data.flatten() > -999)])
april_projections.describe().round(2)

count    2498785.00
mean         116.49
std           20.91
min           49.56
25%          100.08
50%          117.50
75%          128.58
max          249.38
dtype: float64

In [16]:
october_image = gdal.Open(october_proj_file, gdal.GA_ReadOnly)
october_data = october_image.ReadAsArray()[0]
october_projections = pd.Series(october_data.flatten()[(october_data.flatten() > -999)])
october_projections.describe().round(2)

count    2498785.00
mean         111.84
std           20.88
min           47.48
25%           98.87
50%          108.40
75%          128.24
max          205.92
dtype: float64

In [17]:
def data_to_series(data, min_value=-999):
    return pd.Series(data.flatten()[(data.flatten() > min_value)])

In [18]:
west_april = data_to_series(april_data[:, :1287])
east_april = data_to_series(april_data[:, 1287:])
west_october = data_to_series(october_data[:, :1287])
east_october = data_to_series(october_data[:, 1287:])

In [77]:
split_point1 = 1544 # 1154
split_point2 = 1544 # 1655  # 1154
west_april = data_to_series(april_data[:, :split_point1])
central_april = data_to_series(april_data[:, split_point1:split_point2])
east_april = data_to_series(april_data[:, split_point2:])
west_october = data_to_series(october_data[:, :split_point1])
central_october = data_to_series(october_data[:, split_point1:split_point2])
east_october = data_to_series(october_data[:, split_point2:])

In [78]:
pd.DataFrame({'conus_april': april_projections.describe(), 'conus_october': october_projections.describe(),
              'west_april': west_april.describe(), 'west_october': west_october.describe(),
              'central_april': central_april.describe(), 'central_october': central_october.describe(),
              'east_april': east_april.describe(), 'east_october': east_october.describe()}).round(2)

,conus_april,conus_october,west_april,west_october,central_april,central_october,east_april,east_october
count,2498785.00,2498785.00,1430150.00,1430150.00,0.0,0.0,1068635.00,1068635.00
mean,116.49,111.84,116.91,100.00,NaN,NaN,115.92,127.68
std,20.91,20.88,23.11,15.86,NaN,NaN,17.53,15.62
min,49.56,47.48,49.56,47.48,NaN,NaN,75.40,90.34
25%,100.08,98.87,100.96,91.77,NaN,NaN,99.28,112.66
50%,117.50,108.40,116.11,101.29,NaN,NaN,118.56,130.32
75%,128.58,128.24,129.20,108.03,NaN,NaN,127.72,140.37
max,249.38,205.92,249.38,205.92,NaN,NaN,199.06,187.41


In [20]:
KOPPEN_FILE = 'Beck_KG_V1_present_0p0083.tif'
# Climate zone source
czone_file = os.path.join(common.SOURCE_DIR, KOPPEN_FILE)
czone_src = gdal.Open(czone_file, gdal.GA_ReadOnly)
czone_proj = czone_src.GetProjection()
czone_geotrans = czone_src.GetGeoTransform()

# Auxiliary projection and resolution
aux_proj = april_image.GetProjection()
aux_geotrans = april_image.GetGeoTransform()
x_size = april_image.RasterXSize
y_size = april_image.RasterYSize

# In-memory raster for the reprojected data
dst = gdal.GetDriverByName('MEM').Create("", x_size, y_size, 1, gdal.GDT_Byte)
dst.SetGeoTransform(aux_geotrans)
dst.SetProjection(aux_proj)

# Reproject climate zone data to auxiliary projection and resolution. Use the mode of the climate zones
gdal.ReprojectImage(czone_src, dst, czone_proj, aux_proj, gdal.GRA_Mode)

czone_data = dst.ReadAsArray()
czone_data.shape

(1383, 3217)

In [21]:
df = pd.DataFrame({'April_data': april_data.flatten(),
                   'October_data': october_data.flatten(),
                   'Climate_zone': czone_data.flatten()})
df = df[df.April_data > -999]

In [22]:
df = df.merge(czones[['Code', 'Description']], left_on="Climate_zone", right_index=True
             ).drop(columns='Climate_zone').rename(columns={'Code': 'Climate_zone'})
df['Difference'] = df.October_data - df.April_data

In [23]:
display_frames([d[1].unstack().T for d in df.drop(columns='Description').groupby('Climate_zone').describe().iterrows()],
               [str(d[0]) for d in df.groupby('Climate_zone').describe().iterrows()],
               precision=2)

,April_data,October_data,Difference
count,309.00,309.00,309.00
mean,164.16,152.92,-11.24
std,5.13,5.31,5.07
min,150.74,140.91,-23.10
25%,161.39,149.24,-14.48
50%,164.02,152.47,-12.18
75%,166.91,155.71,-8.38
max,181.15,172.18,5.58
,April_data,October_data,Difference
count,2075.00,2075.00,2075.00


In [24]:
df.groupby(['Climate_zone', 'Description']).median().astype(int).sort_values('Difference')

,,April_data,October_data,Difference
Climate_zone,Description,,,
Dsa,"Cold, dry summer, hot summer",163,89,-73
Csa,"Temperate, dry summer, hot summer",117,69,-48
BSh,"Arid, steppe, hot",162,125,-35
BWk,"Arid, desert, cold",130,97,-33
Aw,"Tropical, savannah",172,157,-15
Am,"Tropical, monsoon",169,155,-14
BSk,"Arid, steppe, cold",114,99,-13
Af,"Tropical, rainforest",164,152,-12
BWh,"Arid, desert, hot",73,61,-11


In [26]:
df['April_diffs'] = april_diffs.set_axis(df.index)
df['October_diffs'] = october_diffs.set_axis(df.index)

In [36]:
df[df.Description.str.contains('Arid') | df.Description.str.contains('dry summer')][['April_data', 'October_data']].median()

April_data      117.968964
October_data     98.923248
dtype: float32

In [37]:
df[~(df.Description.str.contains('Arid') | df.Description.str.contains('dry summer'))][['April_data', 'October_data']].median()

April_data      117.304665
October_data    118.929665
dtype: float32

In [75]:
df[df.Description.str.contains('Arid')
   | df.Description.str.contains('Tropical')
   | df.Description.str.contains('Polar')
   | df.Description.str.contains('dry summer')
  ][['April_data', 'October_data']].mean()

April_data      119.672554
October_data     98.495506
dtype: float32

In [74]:
df[~(df.Description.str.contains('Arid')
   | df.Description.str.contains('Tropical')
   | df.Description.str.contains('Polar')
   | df.Description.str.contains('dry summer'))
  ][['April_data', 'October_data']].mean()

April_data      114.118591
October_data    121.782051
dtype: float32

In [72]:
czones[czones.Description.str.contains('Arid') 
       | czones.Description.str.contains('Tropical')
       | czones.Description.str.contains('Polar')
       | czones.Description.str.contains('dry summer')]

,Code,Description,Red,Green,Blue
Number,,,,,
1,Af,"Tropical, rainforest",0,0,255
2,Am,"Tropical, monsoon",0,120,255
3,Aw,"Tropical, savannah",70,170,250
4,BWh,"Arid, desert, hot",255,0,0
5,BWk,"Arid, desert, cold",255,150,150
6,BSh,"Arid, steppe, hot",245,165,0
7,BSk,"Arid, steppe, cold",255,220,100
8,Csa,"Temperate, dry summer, hot summer",255,255,0
9,Csb,"Temperate, dry summer, warm summer",200,200,0


In [73]:
czones[~(czones.Description.str.contains('Arid') 
       | czones.Description.str.contains('Tropical')
       | czones.Description.str.contains('Polar')
       | czones.Description.str.contains('dry summer'))]

,Code,Description,Red,Green,Blue
Number,,,,,
11,Cwa,"Temperate, dry winter, hot summer",150,255,150
12,Cwb,"Temperate, dry winter, warm summer",100,200,100
13,Cwc,"Temperate, dry winter, cold summer",50,150,50
14,Cfa,"Temperate, no dry season, hot summer",200,255,80
15,Cfb,"Temperate, no dry season, warm summer",100,255,80
16,Cfc,"Temperate, no dry season, cold summer",50,200,0
21,Dwa,"Cold, dry winter, hot summer",170,175,255
22,Dwb,"Cold, dry winter, warm summer",90,120,220
23,Dwc,"Cold, dry winter, cold summer",75,80,180


In [42]:
xform = april_image.GetGeoTransform()

In [43]:
xform[0] + xform[1] * 1287

-101.61742493960028

In [44]:
(-104 - xform[0]) / xform[1]

1154.386478749775

In [64]:
xform[0] + xform[1] * 1154

-104.00694359535821

In [67]:
(-95 - xform[0]) / xform[1]

1655.324187319506

In [76]:
(-97 - xform[0]) / xform[1]

1544.0046965262325